# Example of selection interace use

This notebook shows how to use the selection interface to define a if/elsif/else workflow with multiple execution paths based on the input conditions.

As first step, let's connect to the Ophidia Server

In [ ]:
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

The JSON object associated to the workflow is shown in the cell below.

The workflow defines three different execution paths:

1. The *if* path consisting of:

  1. single **import** operator which performs also the subset according to the ```$5``` argument. The task has a flow dependency from the **IF block** task
  
2. The *elseif* path consisting of:

  1. **createcontainer** operator used to create a working container for a random datacube. The task has a flow dependency from the **ELSEIF block** task
  2. **randucube** operator to create a datacube with random values. This task has a flow dependency on the previous task
  
3. The *else* path consisting of:

  1. **import** operator to read the whole file, which has a flow dependency from the **ELSE block** task
  2. **subset** operator to extract a subset of the imported file according to the ```$5``` argument. The task a **single** dependency from the **Import** task since it exploits only one output of parent task

The **Selection block end** closes the various if/elseif/else branches. It has a three **single** dependencies from the last task of each branch in order to make the datacube identifiers available outside the scope of the conditional construct.

In [ ]:
workflow_if_endif_else = """{
        "name": "Selection Interface",
        "author": "CMCC",
        "abstract": "Selection statement with two selection blocks",
        "exec_mode": "sync",
        "ncores": "1",
        "tasks":
        [
                {
                        "name": "IF block",
                        "operator": "oph_if",
                        "arguments": [ "condition=$1" ]
                },
                {
                        "name": "Import and subset",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=$3",
                                "measure=$4",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Max Temp imported and subsetted",
                                "subset_dims=lat|lon|time",
                                "subset_filter=$5",
                                "subset_type=coord"
                        ],
                        "dependencies":
                        [
                                { "task": "IF block" }
                        ]
                },
            
                {
                        "name": "ELSEIF block",
                        "operator": "oph_elseif",
                        "arguments": [ "condition=$2" ],
                        "dependencies":
                        [
                                { "task": "IF block" }
                        ]
                },
                {
                        "name": "Create container",
                        "operator": "oph_createcontainer",
                        "on_error": "skip",
                        "arguments": 
                        [
                                "container=random",
                                "dim=lat|lon|time",
                                "dim_type=double|double|double",
                                "hierarchy=oph_base|oph_base|oph_time"
                        ],
                        "dependencies":
                        [
                                { "task": "ELSEIF block" }
                        ]
                },
                {
                        "name": "Randcube",
                        "operator": "oph_randcube",
                        "arguments":
                        [
                                "container=random",
                                "measure_type=double",
                                "exp_ndim=2",
                                "dim=lat|lon|time",
                                "concept_level=c|c|d",
                                "dim_size=16|10|10",
                                "compressed=yes",
                                "measure=test_var",
                                "description=random cube",
                                "nhost=1",
                                "nfrag=16",
                                "ntuple=10"
                        ],
                        "dependencies":
                        [
                                { "task": "Create container" }
                        ]
                },
                
                {
                        "name": "ELSE block",
                        "operator": "oph_else",
                        "arguments": [  ],
                        "dependencies":
                        [
                                { "task": "ELSEIF block" }
                        ]
                },
                {
                        "name": "Import data",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=$3",
                                "measure=$4",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Max Temp imported"
                        ],
                        "dependencies":
                        [
                                { "task": "ELSE block" }
                        ]
                },
                {
                        "name": "Subset data",
                        "operator": "oph_subset",
                        "arguments":
                        [
                               "subset_dims=lat|lon|time",
                               "subset_filter=$5",
                               "subset_type=coord",
                               "description=Max Temp subsetted" 
                        ],
                        "dependencies":
                        [
                                { "task": "Import data",
                                  "type": "single" }
                        ]
                },
                {
                        "name": "Selection block end",
                        "operator": "oph_endif",
                        "arguments": [ ],
                        "dependencies":
                        [
                                { "task": "Subset data"},
                                { "task": "Import and subset" },
                                { "task": "Randcube" }
                        ]
                }
        ]
}"""

Let's define the workflow arguments:
- *if condition* ```$1```
- *elseif condition* ```$2```
- *NetCDF filename* ```$3```
- *variable* to be imported ```$4```
- *subset filter* (lat|lon|time) ```$5```

In [ ]:
file="/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
variable="tasmax"
lat_lon_time="-50:10|20:140|150:240"

First case: new datacube randomly generated inside a container named "workflow"
    
    - if condition = 0
    - elseif condition = 1
    
 <img src="../imgs/Selection_Interface_randcube.svg" alt="Selection_Interface_randcube" width="200">

In [ ]:
if_cond=0
elseif_cond=1

Run the workflow

In [ ]:
cube.Cube.client.wsubmit(workflow_if_endif_else, if_cond, elseif_cond, file, variable,lat_lon_time)

In [ ]:
cube.Cube.list(level=2)

In [ ]:
cube.Cube.deletecontainer(container="random",force='yes')

Second case:  datacube first imported, then subsetted
    
    - if condition = 0
    - elseif condition = 0
    
 <img src="../imgs/Selection_Interface_import+subset.svg" alt="Selection_Interface_import+subset" width="200">

In [ ]:
if_cond=0
elseif_cond=0

In [ ]:
cube.Cube.client.wsubmit(workflow_if_endif_else, if_cond, elseif_cond, file, variable,lat_lon_time)

In [ ]:
cube.Cube.list(level=2)

In [ ]:
cube.Cube.deletecontainer(container="tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc",force='yes')

Third case: datacube imported and subsetted at the same time
    
    - if condition = 1
    - elseif condition = X
    
 <img src="../imgs/Selection_Interface_importANDsubset.svg" alt="Selection_Interface_importANDsubset" width="200">

In [ ]:
if_cond=1
elseif_cond=0

In [ ]:
cube.Cube.client.wsubmit(workflow_if_endif_else, if_cond, elseif_cond, file, variable,lat_lon_time)

In [ ]:
cube.Cube.list(level=2)

In [ ]:
cube.Cube.deletecontainer(container="tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc",force='yes')